In [ ]:
import matplotlib.dates as md

In [ ]:
# %load ../batchedRNN/model/Data.py
import torch
import numpy as np
from torch.utils import data
 
class DataLoader(object):
    def __init__(self, xs, ys, batch_size, pad_with_last_sample=True, shuffle=True):
        self.batch_size = batch_size
        self.current_ind = 0
        if pad_with_last_sample:
            num_padding = (batch_size - (len(xs) % batch_size)) % batch_size
            x_padding = np.repeat(xs[-1:], num_padding, axis=0)
            y_padding = np.repeat(ys[-1:], num_padding, axis=0)
            xs = np.concatenate([xs, x_padding], axis=0)
            ys = np.concatenate([ys, y_padding], axis=0)
        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        self.xs = xs
        self.ys = ys
        if shuffle:
            self.shuffle()

    def get_iterator(self):
        self.current_ind = 0

        def _wrapper():
            while self.current_ind < self.num_batch:
                start_ind = self.batch_size * self.current_ind
                end_ind = min(self.size, self.batch_size * (self.current_ind + 1))
                x_i = np.transpose(self.xs[start_ind: end_ind, ...], (1,0,3,2))
                y_i = np.transpose(self.ys[start_ind: end_ind, :,:,0], (1,0,2))
                yield (x_i, y_i)
                self.current_ind += 1

        return _wrapper()


    def shuffle(self):
        permutation = np.random.permutation(self.size)
        self.xs, self.ys = self.xs[permutation], self.ys[permutation]

class DataLoaderWithTime(object):
    def __init__(self, xs, ys, tx, ty, batch_size, pad_with_last_sample=True, shuffle=True):
        self.batch_size = batch_size
        self.current_ind = 0
        if pad_with_last_sample:
            num_padding = (batch_size - (len(xs) % batch_size)) % batch_size
            x_padding = np.repeat(xs[-1:], num_padding, axis=0)
            y_padding = np.repeat(ys[-1:], num_padding, axis=0)
            tx_padding = np.repeat(tx[-1:], num_padding, axis=0)
            ty_padding = np.repeat(ty[-1:], num_padding, axis=0)
            xs = np.concatenate([xs, x_padding], axis=0)
            ys = np.concatenate([ys, y_padding], axis=0)
            tx = np.concatenate([tx, tx_padding], axis=0)
            ty = np.concatenate([ty, ty_padding], axis=0)
        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        self.xs = xs
        self.ys = ys
        self.tx = tx
        self.ty = ty
        if shuffle:
            self.shuffle()

    def get_iterator(self):
        self.current_ind = 0

        def _wrapper():
            while self.current_ind < self.num_batch:
                start_ind = self.batch_size * self.current_ind
                end_ind = min(self.size, self.batch_size * (self.current_ind + 1))
                x_i = np.transpose(self.xs[start_ind: end_ind, ...], (1,0,3,2))
                y_i = np.transpose(self.ys[start_ind: end_ind, :,:,0], (1,0,2))
                tx_i = np.transpose(self.tx[start_ind: end_ind, ...], (1,0))
                ty_i = np.transpose(self.ty[start_ind: end_ind, ...], (1,0))
                yield (x_i, y_i, tx_i, ty_i)
                self.current_ind += 1

        return _wrapper()

    def shuffle(self):
        permutation = np.random.permutation(self.size)
        self.xs, self.ys, self.tx, self.ty = self.xs[permutation], self.ys[permutation], self.tx[permutation], self.ty[permutation]


In [ ]:
# %load ../batchedRNN/utils.py
import logging, sys
import torch
import h5py
import os
import numpy as np
import torch.utils.data as torchUtils
import torch.optim as optim
from functools import partial
import torch.nn as nn
import json
from shutil import copy2, copyfile, copytree
import argparse
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

# parser = argparse.ArgumentParser(description='Batched Sequence to Sequence')
# parser.add_argument('--h_dim', type=int, default=256)
# parser.add_argument("--z_dim", type=int, default=128)
# parser.add_argument('--no_cuda', action='store_true', default=False,
#                                         help='disables CUDA training')
# parser.add_argument("--no_attn", action="store_true", default=True, help="Do not use AttnDecoder")
# parser.add_argument("--n_epochs", type=int, default=200)
# parser.add_argument("--batch_size", type=int, default= 64)
# parser.add_argument("--n_layers", type=int, default=2)
# parser.add_argument("--initial_lr", type=float, default=1e-4)
# parser.add_argument("--lr_decay_every", type=int, default=10)
# parser.add_argument("--lr_decay_factor", type=float, default=.10)
# parser.add_argument("--lr_decay_beginning", type=int, default=20)
# parser.add_argument("--print_every", type=int, default = 200)
# parser.add_argument("--criterion", type=str, default="L1Loss")
# parser.add_argument("--save_freq", type=int, default=10)
# parser.add_argument("--down_sample", type=float, default=0.0, help="Keep this fraction of the training data")
# # parser.add_argument("--data_dir", type=str, default="./data/reformattedTraffic/")
# parser.add_argument("--model", type=str, default="sketch-rnn")
# parser.add_argument("--lambda_l1", type=float, default=0)
# parser.add_argument("--lambda_l2", type=float, default=5e-4)
# parser.add_argument("--no_schedule_sampling", action="store_true", default=False)
# parser.add_argument("--scheduling_start", type=float, default=1.0)
# parser.add_argument("--scheduling_end", type=float, default=0.0)
# parser.add_argument("--tries", type=int, default=12)
# parser.add_argument("--kld_warmup_until", type=int, default=5)
# parser.add_argument("--kld_weight_max", type=float, default=0.10)
# parser.add_argument("--no_shuffle_after_epoch", action="store_true", default=False)
# parser.add_argument("--clip", type=int, default=10)
# parser.add_argument("--dataset", type=str, default="traffic")
# parser.add_argument("--predictOnTest", action="store_true", default=True)
# parser.add_argument("--encoder_input_dropout", type=float, default=0.5)
# parser.add_argument("--encoder_layer_dropout", type=float, default=0.5)
# parser.add_argument("--decoder_input_dropout", type=float, default=0.5)
# parser.add_argument("--decoder_layer_dropout", type=float, default=0.5)
# parser.add_argument("--noEarlyStopping", action="store_true", default=False)
# parser.add_argument("--earlyStoppingPatients", type=int, default=3)
# parser.add_argument("--earlyStoppingMinDelta", type=float, default=0.0001)
# parser.add_argument("--bidirectionalEncoder", type=bool, default=True)
# parser.add_argument("--local", action="store_true", default=False)
# parser.add_argument("--debugDataset", action="store_true", default=False)
# parser.add_argument("--encoder_h_dim", type=int, default=256)
# parser.add_argument("--decoder_h_dim", type=int, default=512)
# parser.add_argument("--num_mixtures", type=int, default=20)
# args = parser.parse_args()
# logging.basicConfig(stream=sys.stderr,level=logging.DEBUG)

def plotLosses(trainLosses, valLosses, trainKLDLosses=None, valKLDLosses=None):
    torch.save(trainLosses, args.save_dir+"plot_train_recon_losses")
    torch.save(valLosses, args.save_dir+"plot_val_recon_losses")
    if trainKLDLosses and valKLDLosses:
        torch.save(trainKLDLosses, args.save_dir+"plot_train_KLD_losses")
        torch.save(valKLDLosses, args.save_dir+"plot_val_KLD_losses")
    plt.rcParams.update({'font.size': 8})
    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel(args.criterion, color="r")
    ax1.tick_params('y', colors='r')
    ax1.plot(np.arange(1, len(trainLosses)+1), trainLosses, "r--", label="train reconstruction loss")
    ax1.plot(np.arange(1, len(valLosses)+1), valLosses, color="red", label="validation reconstruction loss")
    ax1.legend(loc="upper left")
    ax1.grid()
    plt.title("Losses for {}".format(args.model))
    plt.savefig(args.save_dir + "train_val_loss_plot.png")

def getSaveDir():
    if args.local:
        saveDir = '../save/local/models/model0/'
    else:
        saveDir = '../save/models/model0/'
    while os.path.isdir(saveDir):
        numStart = saveDir.rfind("model")+5
        numEnd = saveDir.rfind("/")
        saveDir = saveDir[:numStart] + str(int(saveDir[numStart:numEnd])+1) + "/"
    os.mkdir(saveDir)
    return saveDir

def saveUsefulData():
    argsFile = args.save_dir + "args.txt"
    with open(argsFile, "w") as f:
        f.write(json.dumps(vars(args)))
    copy2("./train.py", args.save_dir+"train.py")
    copy2("./utils.py", args.save_dir+"utils.py")
    copy2("./gridSearchOptimize.py", args.save_dir+"gridsearchOptimize.py")
    copytree("./model", args.save_dir+"model/")

def getTrafficDataset(dataDir, category):
    f = np.load(os.path.join(dataDir, category + '.npz'))
    my_dataset = torchUtils.TensorDataset(torch.Tensor(f["inputs"]),torch.Tensor(f["targets"])) # create your datset
    scaler = getScaler(f["inputs"])
    sequence_len = f['inputs'].shape[1]
    x_dim = f['inputs'].shape[2]
    channels = f["inputs"].shape[3]
    return my_dataset, scaler, sequence_len, sequence_len, x_dim, channels

def getHumanDataset(dataDir, category):
    f = h5py.File(os.path.join(dataDir, category+".h5"), "r")
    my_dataset = torchUtils.TensorDataset(torch.Tensor(f["input2d"]), torch.Tensor(f["target2d"]))
    scaler = getScaler(f["input2d"])
    input_sequence_len = f["input2d"].shape[1]
    target_sequence_len = f["target2d"].shape[1]
    x_dim = f["input2d"].shape[2]
    channels = f["input2d"].shape[3]
    return my_dataset, scaler, input_sequence_len, target_sequence_len, x_dim, channels

def getLoaderAndScaler(dataDir, category):
    logging.info("Getting {} loader".format(category))
    if args.dataset == "traffic":
        my_dataset, scaler, input_sequence_len, target_sequence_len, x_dim, channels = getTrafficDataset(dataDir, category)
    else:
        my_dataset, scaler, input_sequence_len, target_sequence_len, x_dim, channels = getHumanDataset(dataDir, category)
    shf = False
    if category == "train":
        shf = True
    loader = torchUtils.DataLoader(
        my_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=False,
        drop_last=True
        )
    return loader, scaler, input_sequence_len, target_sequence_len, x_dim, channels # create your dataloader

def getDataLoaders(dataDir, debug=False):
    loaders = {}
    logging.info("Getting data from {}".format(dataDir))
    if debug:
        categories = ["test"]
        scalerSet = "test"
    else:
        categories = ["train", "val", "test"]
        scalerSet = "train"
    for category in categories:
        loader, scaler, input_sequence_len, target_sequence_len, x_dim, channels = getLoaderAndScaler(dataDir, category)
        if category == scalerSet:
            loaders["scaler"] = scaler
            loaders["input_sequence_len"] = input_sequence_len
            loaders["target_sequence_len"] = target_sequence_len
            loaders["x_dim"] = x_dim
            loaders["channels"] = channels
        loaders[category] = loader
    return loaders

class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean0, std0, mean1, std1):
        self.mean0 = mean0
        self.std0 = std0
        self.mean1 = mean1
        self.std1 = std1

    def transform(self, data):
        mean = torch.zeros(data.size())
        mean[...,0] = self.mean0
        mean[...,1] = self.mean1
        std = torch.ones(data.size())
        std[...,0] = self.std0
        std[...,1] = self.std1
        return torch.div(torch.sub(data,mean),std)

class StandardScalerTraffic(StandardScaler):
    def __init__(self, mean0, std0):
        super(StandardScalerTraffic, self).__init__(mean0, std0, 0.0, 1.0)

    def inverse_transform(self, data, permute=True):
        """
        Inverse transform is applied to output and target.
        These are only the speeds, so only use the first 
        """
        mean = torch.ones(data.size()) * self.mean0
        std = torch.ones(data.size()) * self.std0
        if args.cuda:
            mean = mean.cuda()
            std = std.cuda()
        transformed = torch.add(torch.mul(data, std), mean)
        del mean, std
        if permute:
            return transformed.permute(1,0,2)
        else:
            return transformed

    def transformBatchForEpoch(self, batch):
        x = self.transform(batch[0]).permute(1,0,3,2)
        y = self.transform(batch[1])[...,0].permute(1,0,2)
        if args.cuda:
            return x.cuda(), y.cuda()
        return x, y

class StandardScalerHuman(StandardScaler):
    """docstring for StandardScalerHuman"""
    def __init__(self, mean0, std0, mean1, std1):
        super(StandardScalerHuman, self).__init__(mean0, std0, mean1, std1)

    def inverse_transform(self, data):
        """
        applied to output and target
        """
        transed = self.restoreDim(data)
        mean = torch.zeros(transed.size())
        std = torch.ones(transed.size())
        if args.cuda:
            mean = mean.cuda()
            std = std.cuda()
        mean[...,0] = self.mean0
        mean[...,1] = self.mean1
        std[...,0] = self.std0
        std[...,1] = self.std1
        transformed =  torch.add(torch.mul(transed, std), mean)
        del mean, std
        return transformed.permute(1,0,3,2)

    def restoreDim(self, data):
        l1, l2 = torch.split(data, int(data.size(2) / 2), 2)
        return torch.cat((l1.unsqueeze(3), l2.unsqueeze(3)), dim=3)

    def removeDim(self, data):
        layer0, layer1 = torch.split(data, 1, dim=3)
        return torch.cat((layer0.squeeze(3), layer1.squeeze(3)), dim=2)

    def transformBatchForEpoch(self, batch):
        x = self.transform(batch[0]).permute(1,0,3,2)
        y = self.transform(batch[1])
        wideY = self.removeDim(y).permute(1,0,2)
        if args.cuda:
            return x.cuda(), wideY.cuda()
        return x, wideY

def getScaler(trainX):
    mean0 = np.mean(trainX[...,0])
    std0 = np.std(trainX[...,0])
    mean1 = np.mean(trainX[...,1])
    std1 = np.std(trainX[...,1])
    if args.dataset == "traffic":
        return StandardScalerTraffic(mean0, std0)
    elif args.dataset == "human":
        return StandardScalerHuman(mean0, std0, mean1, std1)
    else:
        assert False, "bad dataset"

def getReconLoss(output, target, scaler):
    output = scaler.inverse_transform(output)
    target = scaler.inverse_transform(target)
    assert output.size() == target.size(), "output size: {}, target size: {}".format(output.size(), target.size())
    if args.criterion == "RMSE":
        criterion = nn.MSELoss()
        return torch.sqrt(criterion(output, target))
    elif args.criterion == "L1Loss":
        criterion = nn.L1Loss()
        return criterion(output, target)
    else:
        assert False, "bad loss function"

def getKLDWeight(epoch):
    # kldLossWeight = args.kld_weight_max * min((epoch / (args.kld_warmup_until)), 1.0)
    kldLossWeight = args.kld_weight_max
    return kldLossWeight

def kld_gauss(mean_1, std_1, mean_2, std_2):
    """Using std to compute KLD"""

    kld_element = (2 * torch.log(std_2) - 2 * torch.log(std_1) +
                   (std_1.pow(2) + (mean_1 - mean_2).pow(2)) /
                   std_2.pow(2) - 1)
    return 0.5 * torch.sum(kld_element)

def sketchRNNKLD(latentMean, latentStd):
    m2 = torch.zeros_like(latentMean)
    s2 = torch.ones_like(latentStd)
    return kld_gauss(latentMean, latentStd, m2, s2)

def getLoss(model, output, target, scaler, epoch):
    if args.model == "rnn":
        reconLoss = getReconLoss(output, target, scaler)
        return reconLoss, 0
    else:
        latentMean, latentStd, z, predOut, predMeanOut, predStdOut = output
        reconLoss = getReconLoss(predOut, target, scaler)
        kldLoss = sketchRNNKLD(latentMean, latentStd)
        return reconLoss, kldLoss

def saveModel(modelWeights, epoch):
    fn = args.save_dir+'{}_state_dict_'.format(args.model)+str(epoch)+'.pth'
    torch.save(modelWeights, fn)
    logging.info('Saved model to '+fn)

class EarlyStoppingObject(object):
    """docstring for EarlyStoppingObject"""
    def __init__(self):
        super(EarlyStoppingObject, self).__init__()
        self.bestLoss = None
        self.bestEpoch = None
        self.counter = 0
        self.epochCounter = 0

    def checkStop(self, previousLoss):
        self.epochCounter += 1
        if not args.noEarlyStopping:
            if self.bestLoss is not None and previousLoss + args.earlyStoppingMinDelta >= self.bestLoss:
                self.counter += 1
                if self.counter >= args.earlyStoppingPatients:
                    logging.info("Stopping Early, haven't beaten best loss {:.4f} @ Epoch {} in {} epochs".format(
                        self.bestLoss,
                        self.bestEpoch,
                        args.earlyStoppingPatients))
                    return True
            else:
                self.bestLoss = previousLoss
                self.bestEpoch = self.epochCounter
                self.counter = 0
                return False

        else:
            return False


In [ ]:
# %load ../batchedRNN/model/RoseSeq2Seq
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Parameter
from torch.autograd import Variable
import numpy as np

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=2, bidirectional=True, args=None):
        super(EncoderRNN, self).__init__()
        self.args = args
        self.input_size = input_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size * self.args.channels, hidden_size, n_layers, dropout=self.args.encoder_layer_dropout, bidirectional=bidirectional)
        self.input_dropout = nn.Dropout(p=self.args.encoder_input_dropout)
    def forward(self, input, hidden):
        embedded = self.embedding(input)
        embedded = self.input_dropout(embedded)
        embedded = torch.unsqueeze(embedded, 0)
        embedded = embedded.view(1, self.args.batch_size, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        if self.args.bidirectionalEncoder:
            directions = 2
        else:
            directions = 1
        result = Variable(torch.zeros(self.n_layers * directions, self.args.batch_size, self.hidden_size))
        if self.args.cuda:
            return result.cuda()
        else:
            return result

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=2, args=None):
        super(DecoderRNN, self).__init__()
        self.args = args

        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Linear(output_size, hidden_size)
        self.input_dropout = nn.Dropout(p=self.args.decoder_input_dropout)
        if self.args.bidirectionalEncoder:
            directions = 2
        else:
            directions = 1
        # encoder hidden is (layers * directions, batch, hidden_size)
        # converted to (layers, batch, hidden_size * directions)
        self.gru = nn.GRU(hidden_size, directions * hidden_size, n_layers, dropout=self.args.decoder_layer_dropout)
        # GRU output (seq_len, batch, directions * hidden_size)
        self.out = nn.Linear(directions * hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        embedded = self.input_dropout(embedded)
        embedded = F.relu(embedded)
        embedded = torch.unsqueeze(embedded, 0)
        output, hidden = self.gru(embedded, hidden)
        output = self.out(output.squeeze(0))
        #print("decoder output", output[10,31])
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, args):
        super(Seq2Seq, self).__init__()
        self.args = args

        self.enc = EncoderRNN(self.args.x_dim, self.args.h_dim, n_layers=self.args.n_layers, bidirectional=args.bidirectionalEncoder, args=args)

        self.dec = DecoderRNN(self.args.h_dim, self.args.output_dim, n_layers=self.args.n_layers, args=args)

        self.use_schedule_sampling = args.use_schedule_sampling
        self.scheduling_start = args.scheduling_start
        self.scheduling_end = args.scheduling_end

    def _cat_directions(self, h):
        """ If the encoder is bidirectional, do the following transformation.
            (#directions * #layers, #batch, hidden_size) -> (#layers, #batch, #directions * hidden_size)
        """
        h = torch.cat([h[0:h.size(0):2], h[1:h.size(0):2]], 2)
        return h

    def parameters(self):
        return list(self.enc.parameters()) + list(self.dec.parameters())

    def scheduleSample(self, epoch):
        eps = max(self.args.scheduling_start - 
            (self.args.scheduling_start - self.args.scheduling_end)* epoch / self.args.n_epochs,
            self.args.scheduling_end)
        return np.random.binomial(1, eps)

    def forward(self, x, target, epoch):
        encoder_hidden = self.enc.initHidden()
        hs = []
        for t in range(self.args.input_sequence_len):
            encoder_output, encoder_hidden = self.enc(x[t].squeeze(), encoder_hidden)
            hs += [encoder_output]
        if self.args.bidirectionalEncoder:
            decoder_hidden = self._cat_directions(encoder_hidden)
        else:
            decoder_hidden = encoder_hidden
        # Prepare for Decoder
        inp = Variable(torch.zeros(self.args.batch_size, self.args.output_dim))
        if self.args.cuda:
            inp = inp.cuda()
        ys = []
        if not self.training:
            sample=0
        else:
            sample = self.scheduleSample(epoch)
        # Decode
        for t in range(self.args.target_sequence_len):
            decoder_output, decoder_hidden = self.dec(inp, decoder_hidden)
            if sample:
                inp = target[t-1]
            else:
                inp = decoder_output
            ys += [decoder_output]
        return torch.cat([torch.unsqueeze(y, dim=0) for y in ys])


In [ ]:
# %load ../batchedRNN/model/SketchRNN.py
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class SketchRNNEncoder(nn.Module):
    def __init__(self, args):
        super(SketchRNNEncoder, self).__init__()
        self.args = args
        if self.args.bidirectionalEncoder:
            self.directions = 2
        else:
            self.directions = 1
        # bidirectional lstm:
        self.lstm = nn.LSTM(self.args.x_dim * self.args.channels, self.args.encoder_h_dim, \
            self.args.n_layers, dropout=self.args.encoder_layer_dropout, bidirectional=self.args.bidirectionalEncoder)
        # create mu and sigma from lstm's last output:
        self.fc_mu = nn.Linear(self.args.n_layers * self.directions * self.args.encoder_h_dim, self.args.z_dim)
        self.fc_sigma = nn.Linear(self.args.n_layers * self.directions * self.args.encoder_h_dim, self.args.z_dim)
        

    def forward(self, input, hidden_cell=None):
        if hidden_cell is None:
            hidden_cell = self.init_hidden_cell()
        _, (hidden, cell) = self.lstm(input, hidden_cell)
        # convert hidden size from (n_layers * directions, batch_size, h_dim)
        #                       to (batch_size, n_layers * directions * h_dim)
        hiddenLayers = torch.split(hidden, 1, 0)
        if self.directions == 2 and self.args.n_layers == 2:
            assert len(hiddenLayers) == 4
        hidden_cat = torch.cat([h.squeeze(0) for h in hiddenLayers], 1)
        mu = self.fc_mu(hidden_cat)
        sigma_hat = self.fc_sigma(hidden_cat)
        sigma = torch.exp(sigma_hat / 2)
        z_size = mu.size()
        if self.args.cuda:
            N = Variable(torch.normal(torch.zeros(z_size),torch.ones(z_size)).cuda())
        else:
            N = Variable(torch.normal(torch.zeros(z_size),torch.ones(z_size)))
        z = mu + sigma*N
        return z, mu, sigma_hat



    def init_hidden_cell(self):
        hidden = Variable(torch.zeros(self.directions * self.args.n_layers, self.args.batch_size, self.args.encoder_h_dim))
        cell = Variable(torch.zeros(self.directions * self.args.n_layers, self.args.batch_size, self.args.encoder_h_dim))
        if self.args.cuda:
            return (hidden.cuda(), cell.cuda())
        else:
            return (hidden, cell)

class SketchRNNDecoder(nn.Module):
    def __init__(self, args):
        super(SketchRNNDecoder, self).__init__()
        self.args = args
        # to init hidden and cell from z:
        self.fc_hc = nn.Linear(self.args.z_dim, 2 * self.args.n_layers * self.args.decoder_h_dim)
        # unidirectional lstm:
        self.lstm = nn.LSTM(self.args.z_dim + self.args.output_dim, self.args.decoder_h_dim, self.args.n_layers, dropout=self.args.decoder_layer_dropout)
        self.muLayer = nn.Linear(self.args.decoder_h_dim, self.args.output_dim * self.args.n_gaussians)
        self.sigmaLayer = nn.Linear(self.args.decoder_h_dim, self.args.output_dim * self.args.n_gaussians)
        self.piLayer = nn.Linear(self.args.decoder_h_dim, self.args.output_dim * self.args.n_gaussians)

    def forward(self, inputs, z, hidden_cell=None):
        if hidden_cell is None:
            layers = torch.split(torch.tanh(self.fc_hc(z)),self.args.decoder_h_dim,1)
            hidden = torch.stack(layers[:int(len(layers) / 2)], dim=0)
            cell = torch.stack(layers[int(len(layers) / 2): ], dim=0)
            hidden_cell = (hidden.contiguous(), cell.contiguous())
        outputs,(hidden,cell) = self.lstm(inputs, hidden_cell)
        # outputs size: (seq_len, batch, num_directions * hidden_size)
        # hidden size: (num_layers * num_directions, batch, hidden_size)
        # cell size: (num_layers * num_directions, batch, hidden_size)
        mu = self.muLayer(outputs).view(-1, self.args.batch_size, self.args.output_dim, self.args.n_gaussians)
        sigma = self.sigmaLayer(outputs).view(-1, self.args.batch_size, self.args.output_dim, self.args.n_gaussians)
        pi = self.piLayer(outputs).view(-1, self.args.batch_size, self.args.output_dim, self.args.n_gaussians)
        pi = F.softmax(pi, 3)
        sigma = torch.exp(sigma)
        return (pi, mu, sigma), (hidden, cell)

class SketchRNN(nn.Module):
    def __init__(self, args):
        super(SketchRNN, self).__init__()
        self.args = args
        if self.args.cuda:
            self.encoder = SketchRNNEncoder(args).cuda()
            self.decoder = SketchRNNDecoder(args).cuda()
        else:
            self.encoder = SketchRNNEncoder(args)
            self.decoder = SketchRNNDecoder(args)

    def scheduleSample(self, epoch):
        eps = max(self.args.scheduling_start - 
            (self.args.scheduling_start - self.args.scheduling_end)* epoch / self.args.self.args.n_epochs,
            self.args.scheduling_end)
        return np.random.binomial(1, eps)

    def generatePred(self, pi, mu, sigma, individual=False):
        if self.args.cuda:
            N = torch.randn(pi.size()).cuda()
            #N = torch.normal(torch.zeros(pi.size()),torch.ones(pi.size()))
        else:
            N = torch.randn(pi.size())
            #N = torch.normal(torch.zeros(pi.size()),torch.ones(pi.size()))
        clusterPredictions = mu + sigma * N
        weightedClusterPredictions = clusterPredictions * pi
        if individual:
            pred = torch.sum(weightedClusterPredictions, dim=1)
        else:
            pred = torch.sum(weightedClusterPredictions, dim=3)
        return pred

    def allSteps(self, target, z):
        sos = self.getStartOfSequence()
        batch_init = torch.cat([sos, target[:-1,...]], 0)
        z_stack = torch.stack([z]*(self.args.target_sequence_len))
        inp = torch.cat([batch_init, z_stack], 2)
        (pi, mu, sigma), (hidden, cell) = self.decoder(inp, z)
        return (pi, mu, sigma)

    def oneStepAtATime(self, z):
        sos = self.getStartOfSequence()
        inp = torch.cat([sos, z.unsqueeze(0)], 2)
        piList, muList, sigmaList = [], [], []
        for timeStep in range(self.args.target_sequence_len):
            (pi, mu, sigma), (hidden, cell) = self.decoder(inp, z)
            pred = self.generatePred(pi, mu, sigma)
            inp = torch.cat([pred, z.unsqueeze(0)], 2)
            piList.append(pi.detach())
            muList.append(mu.detach())
            sigmaList.append(sigma.detach())
        Pi = torch.cat(piList, 0)
        Mu = torch.cat(muList, 0)
        Sigma = torch.cat(sigmaList, 0)
        return (Pi, Mu, Sigma)

    def getStartOfSequence(self):
        if self.args.cuda:
            return Variable(torch.zeros(1, self.args.batch_size, self.args.output_dim).cuda())
        else:
            return Variable(torch.zeros(1, self.args.batch_size, self.args.output_dim))

    def doEncoding(self,batch):
        # convert input from [input_sequence_len, batch_size, channels, x_dim]
        #                 to [input_sequence_len, batch_size, channels * x_dim]
        embedded = batch.contiguous().view(-1, self.args.batch_size, self.args.x_dim * self.args.channels)
        z, mu, sigma_hat = self.encoder(embedded)
        return z, mu, sigma_hat

    def forward(self, batch, target, epoch):
        z, latentMean, latentStd = self.doEncoding(batch)
        if self.training:
            (Pi, Mu, Sigma) = self.allSteps(target, z)
        else:
            (Pi, Mu, Sigma) = self.oneStepAtATime(z)
        return Pi, Mu, Sigma, latentMean, latentStd


In [ ]:
# %load ../batchedRNN/model/encoders.py
from torch import nn
from torch.nn import init
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class RecurrentEncoder(nn.Module):

    def __init__(self, rnn_type, input_dim, hidden_dim,
                 dropout_prob, bidirectional, num_layers):
        super().__init__()
        self.rnn_type = rnn_type
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.dropout_prob = dropout_prob
        self.bidirectional = bidirectional
        self.num_layers = num_layers

        self.dropout = nn.Dropout(dropout_prob)
        self.embedding_layer = nn.Linear(in_features=self.input_dim,
                                           out_features=self.input_dim)
        if num_layers == 1:
            RNNDROPOUT = 0.0
        else:
            RNNDROPOUT = dropout_prob
        if rnn_type == 'gru':
            self.rnn = nn.GRU(
                input_size=input_dim, hidden_size=hidden_dim,
                bidirectional=bidirectional, num_layers=num_layers,
                dropout=RNNDROPOUT)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(
                input_size=input_dim, hidden_size=hidden_dim,
                bidirectional=bidirectional, num_layers=num_layers,
                dropout=RNNDROPOUT)
        else:
            raise ValueError('Unknown RNN type!')
        self.reset_parameters()

    def reset_parameters(self):
        init.normal_(self.embedding_layer.weight.data, mean=0, std=0.01)
        for i in range(self.num_layers):
            suffixes = ['']
            if self.bidirectional:
                suffixes.append('_reverse')
            for suffix in suffixes:
                weight_ih = getattr(self.rnn, f'weight_ih_l{i}{suffix}')
                weight_hh = getattr(self.rnn, f'weight_hh_l{i}{suffix}')
                bias_ih = getattr(self.rnn, f'bias_ih_l{i}{suffix}')
                bias_hh = getattr(self.rnn, f'bias_hh_l{i}{suffix}')
                init.orthogonal_(weight_hh.data)
                init.kaiming_normal_(weight_ih.data)
                init.constant_(bias_ih.data, val=0)
                init.constant_(bias_hh.data, val=0)
                if self.rnn_type == 'lstm':  # Set initial forget bias to 1
                    bias_ih.data.chunk(4)[1].fill_(1)

    def forward(self, encoder_inputs):
        encoder_inputs_emb = self.embedding_layer(encoder_inputs)
        encoder_inputs_emb = self.dropout(encoder_inputs_emb)
        encoder_hidden_states, rnn_state = self.rnn(encoder_inputs_emb)
        # For LSTM, encoder_states does not contain cell states.
        # Thus it is necessary to explicitly return the last state
        encoder_state = rnn_state
        return encoder_hidden_states, encoder_state


In [ ]:
# %load ../batchedRNN/model/basic.py
import torch
from torch.nn import functional


def sequence_mask(length, max_length=None):
    """
    Args:
        length (Tensor): A long tensor of size (batch_size,).
        max_length (int): The maximum length. If None, it automatically
            sets this as max(lengths).
    Returns:
        mask: (Tensor): A byte mask tensor of size
            (batch_size, max_length). Each element is 1 if valid
            and 0 else.
    """

    if max_length is None:
        max_length = length.max()
    seq_range = torch.arange(0, max_length).unsqueeze(0).type_as(length)
    length = length.unsqueeze(1)
    mask = torch.lt(seq_range, length)
    return mask


def sequence_cross_entropy(logits, targets, length):
    logits_flat = logits.view(-1, logits.size(-1))
    targets_flat = targets.view(-1)
    losses_flat = functional.cross_entropy(
        input=logits_flat, target=targets_flat, reduce=False)
    losses = losses_flat.view(*targets.size())
    mask = sequence_mask(length=length, max_length=logits.size(0)).t()
    losses.data.masked_fill_(mask=~mask, value=0)
    loss = losses.sum() / losses.size(1)
    return loss


In [ ]:
# %load ../batchedRNN/model/attention.py
import torch
from torch import nn
from torch.nn import init, functional

# from . import basic


class DotAttention(nn.Module):

    def __init__(self, hidden_dim, dropout_prob):
        super().__init__()

        self.dropout = nn.Dropout(dropout_prob)
        self.attend = nn.Sequential(nn.Linear(in_features=2 * hidden_dim,
            out_features=hidden_dim),
            nn.Tanh())
        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_normal_(self.attend[0].weight.data)
        init.constant_(self.attend[0].bias.data, val=0)

    def score(self, queries, annotations):
        """
        Args:
            queries (Variable): Query vectors to annotations,
                (batch_size, tgt_len, hidden_dim).
            annotations (Variable): Encoded source vectors,
                (batch_size, src_len, hidden_dim).

        Returns:
            scores (Variable): Alignment vectors,
                (batch_size, tgt_len, src_len).
        """

        scores = torch.bmm(queries, annotations.transpose(1, 2))
        scores = functional.softmax(scores, dim=2)
        return scores

    def forward(self, queries, annotations):
        """
        Args:
            queries (Variable): Query vectors to annotations,
                (tgt_len, batch_size, hidden_dim).
            annotations (Variable): Encoded source vectors,
                (src_len, batch_size, hidden_dim).

        Returns:
            attentional_states (Variable): Hidden states after
                being attended, (tgt_len, batch_size, hidden_dim).
            scores (Variable): Alignment vectors,
                (tgt_len, batch_size, src_len).
        """

        # Make queries and annotations batch-major.
        queries = queries.permute(1,0,2)
        annotations = annotations.permute(1,0,2)
        scores = self.score(queries=queries, annotations=annotations)
        contexts = torch.bmm(scores, annotations)
        attention_features = [queries, contexts]
        attention_input = torch.cat(attention_features, dim=2)
        attention_input = self.dropout(attention_input)
        attentional_states = self.attend(attention_input)

        # Convert back to time-major.
        attentional_states = attentional_states.permute(1,0,2)
        scores = scores.permute(1,0,2)
        return attentional_states, scores


class MLPAttention(nn.Module):

    def __init__(self, hidden_dim, annotation_dim, dropout_prob):
        super().__init__()

        self.dropout = nn.Dropout(dropout_prob)
        self.query_linear = nn.Linear(in_features=hidden_dim,
                                      out_features=hidden_dim)
        self.annotation_linear = nn.Linear(in_features=annotation_dim,
                                           out_features=hidden_dim,
                                           bias=False)
        self.tanh = nn.Tanh()
        self.v = nn.Linear(in_features=hidden_dim, out_features=1,
                           bias=False)
        self.attend = nn.Sequential(
            nn.Linear(in_features=hidden_dim + annotation_dim,
                      out_features=hidden_dim),
            nn.Tanh())
        self.reset_parameters()

    def reset_parameters(self):
        init.normal_(self.query_linear.weight.data, mean=0, std=0.001)
        init.constant_(self.query_linear.bias.data, val=0)
        init.normal_(self.annotation_linear.weight.data, mean=0, std=0.001)
        init.constant_(self.v.weight.data, val=0)
        init.kaiming_normal_(self.attend[0].weight.data)
        init.constant_(self.attend[0].bias.data, val=0)

    def score(self, queries, annotations):
        """
        Args:
            queries (Variable): Query vectors to annotations,
                (batch_size, tgt_len, hidden_dim).
            annotations (Variable): Encoded source vectors,
                (batch_size, src_len, annotation_dim).

        Returns:
            scores (Variable): Alignment vectors,
                (batch_size, tgt_len, src_len).
        """

        # queries_proj: (batch_size, tgt_len, hidden_dim)
        queries_proj = self.query_linear(queries)
        # annotations_proj: (batch_size, src_len, hidden_dim)
        annotations_proj = self.annotation_linear(annotations)
        pre_tanh = queries_proj.unsqueeze(2) + annotations_proj.unsqueeze(1)
        scores = self.v(self.tanh(pre_tanh)).squeeze(3)
        scores = functional.softmax(scores, dim=2)
        return scores

    def forward(self, queries, annotations):
        """
        Args:
            queries (Variable): Query vectors to annotations,
                (tgt_len, batch_size, hidden_dim).
            annotations (Variable): Encoded source vectors,
                (src_len, batch_size, hidden_dim).

        Returns:
            attentional_states (Variable): Hidden states after
                being attended, (tgt_len, batch_size, hidden_dim).
            scores (Variable): Alignment vectors,
                (tgt_len, batch_size, src_len).
        """

        # Make queries and annotations batch-major.
        queries = queries.permute(1,0,2)
        annotations = annotations.permute(1,0,2)
        scores = self.score(queries=queries, annotations=annotations)
        contexts = torch.bmm(scores, annotations)
        attention_features = [queries, contexts]
        attention_input = torch.cat(attention_features, dim=2)
        attention_input = self.dropout(attention_input)
        attentional_states = self.attend(attention_input)

        # Convert back to time-major.
        attentional_states = attentional_states.permute(1,0,2)
        scores = scores.permute(1,0,2)
        return attentional_states, scores


In [ ]:
# %load ../batchedRNN/model/decoders.py
import copy

import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import init
import numpy as np


class RecurrentDecoder(nn.Module):

    def __init__(self, rnn_type, input_dim, hidden_dim,
                 annotation_dim, num_layers, attention_type, input_feeding,
                 dropout_prob, args):
        super().__init__()
        self.rnn_type = rnn_type
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.annotation_dim = annotation_dim
        self.num_layers = num_layers
        self.input_feeding = input_feeding
        self.dropout_prob = dropout_prob
        self.args = args
        self.attention_type = attention_type
        self.dropout = nn.Dropout(dropout_prob)
        self.word_embedding = nn.Linear(in_features=self.input_dim,
                                        out_features=self.input_dim)
        rnn_input_size = input_dim
        if input_feeding and attention_type != "None":
            rnn_input_size += hidden_dim
        if num_layers == 1:
            RNNDROPOUT = 0.0
        else:
            RNNDROPOUT = self.dropout_prob
        if rnn_type == 'gru':
            self.rnn = nn.GRU(
                input_size=rnn_input_size, hidden_size=hidden_dim,
                num_layers=num_layers, dropout=RNNDROPOUT)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(
                input_size=rnn_input_size, hidden_size=hidden_dim,
                num_layers=num_layers, dropout=RNNDROPOUT)
        else:
            raise ValueError('Unknown RNN type!')
        if attention_type == 'dot':
            assert hidden_dim == annotation_dim, (
                'hidden_dim and annotation_dim must be same when using'
                ' dot attention.')
            self.attention = DotAttention(
                hidden_dim=hidden_dim, dropout_prob=dropout_prob)
        elif attention_type == 'mlp':
            self.attention = MLPAttention(
                hidden_dim=hidden_dim, annotation_dim=annotation_dim,
                dropout_prob=dropout_prob)
        elif attention_type == "None":
            self.attention = nn.Sequential(nn.Linear(in_features=hidden_dim,
            out_features=hidden_dim),
            nn.Tanh())
        else:
            raise ValueError('Unknown attention type!')
        self.output_linear = nn.Linear(in_features=hidden_dim,
                                       out_features=input_dim)
        self.reset_parameters()
        self.resizeEncoderState = nn.Linear(in_features=hidden_dim * 2, out_features=hidden_dim)

    def reset_parameters(self):
        init.normal_(self.word_embedding.weight.data, mean=0, std=0.01)
        for i in range(self.num_layers):
            weight_ih = getattr(self.rnn, f'weight_ih_l{i}')
            weight_hh = getattr(self.rnn, f'weight_hh_l{i}')
            bias_ih = getattr(self.rnn, f'bias_ih_l{i}')
            bias_hh = getattr(self.rnn, f'bias_hh_l{i}')
            init.orthogonal_(weight_hh.data)
            init.kaiming_normal_(weight_ih.data)
            init.constant_(bias_ih.data, val=0)
            init.constant_(bias_hh.data, val=0)
            if self.rnn_type == 'lstm':  # Set initial forget bias to 1
                bias_ih.data.chunk(4)[1].fill_(1)
        if self.attention_type != "None":
            self.attention.reset_parameters()
        init.normal_(self.output_linear.weight.data, mean=0, std=0.01)
        init.constant_(self.output_linear.bias.data, val=0)

    def scheduleSample(self, epoch):
        eps = max(self.args.scheduling_start - 
            (self.args.scheduling_start - self.args.scheduling_end)* epoch / self.args.n_epochs,
            self.args.scheduling_end)
        return np.random.binomial(1, eps)

    def forward(self, annotations, targets, state, epoch):
        """
        Args:
            annotations (Variable): A float variable of size
                (max_src_length, batch_size, context_dim).
            decoder_inputs (Variable): A long variable of size
                (seq_length, batch_size, n_features) that either the previous
                target or prediction.
            state (DecoderState): The current state of the decoder.

        Returns:
            logits (Variable): A float variable containing unnormalized
                log probabilities. It has the same size as words.
            state (DecoderState): The updated state of the decoder.
            attention_weights (Variable): A float variable of size
                (length, batch_size, max_src_length), which contains
                the attention weight for each time step of the context.
        """
        inp = torch.zeros(targets[0].size())
        if self.args.cuda:
            inp = inp.cuda()
        decoder_inputs = torch.cat([inp.unsqueeze(0), targets[:-1]], dim=0)
        state = copy.copy(state)
        bidirectionalEncoder = state.prepForDecoder(num_layers=self.num_layers)
        if bidirectionalEncoder:
            if isinstance(state.rnn, tuple):
                hidden = self.resizeEncoderState(state.rnn[0])
                cell = self.resizeEncoderState(state.rnn[1])
                state.update_rnn_state((hidden, cell))
            else:
                state.update_rnn_state(self.resizeEncoderState(state.rnn))
        if not self.input_feeding:
            embedded_inputs = self.word_embedding(decoder_inputs)
            embedded_inputs = self.dropout(embedded_inputs)
            rnn_outputs, rnn_state = self.rnn(input=embedded_inputs, hx=state.rnn)
            if self.attention_type != "None":
                attentional_states, attention_weights = self.attention(
                    queries=rnn_outputs, annotations=annotations)
            else:
                attentional_states = rnn_outputs
                attention_weights = torch.zeros((1,1))
                if self.args.cuda:
                    attention_weights = attention_weights.cuda()
            state.update(rnn_state=rnn_state)
            attentional_states = self.output_linear(attentional_states)
        else:
            target_seq_len, batch_size, _ = decoder_inputs.size()
            attentional_states = []
            attention_weights = []
            if state.attention is None:
                zero_attentional_state = torch.zeros((batch_size, self.hidden_dim))
                if self.args.cuda:
                    zero_attentional_state = zero_attentional_state.cuda()
                # zero_attentional_state = (
                #     embedded_inputs.data.new(batch_size, self.hidden_dim).zero_())
                zero_attentional_state = Variable(zero_attentional_state)
                state.update_attentional_state(zero_attentional_state)
            if self.args.no_schedule_sampling or not self.training:
                sample=0
            else:
                sample = self.scheduleSample(epoch)
            for t in range(target_seq_len):
                inp = self.word_embedding(inp)
                inp = self.dropout(inp)
                if self.attention_type != "None":
                    decoder_input_t = torch.cat([inp, state.attention], dim=1)
                    decoder_input_t = decoder_input_t.unsqueeze(0)
                    rnn_output_t, rnn_state_t = self.rnn(
                        input=decoder_input_t, hx=state.rnn)
                    attentional_state_t, attention_weights_t = self.attention(
                        queries=rnn_output_t, annotations=annotations)
                else:
                    attentional_state_t, rnn_state_t = self.rnn(
                        input=inp.unsqueeze(0), hx=state.rnn)
                    attention_weights_t = torch.zeros((1,1))
                    if self.args.cuda:
                        attention_weights_t = attention_weights_t.cuda()
                attentional_state_t = attentional_state_t.squeeze(0)
                state.update_state(rnn_state=rnn_state_t,
                                   attentional_state=attentional_state_t)
                attentional_state_t = self.output_linear(attentional_state_t)
                attentional_states.append(attentional_state_t)
                attention_weights.append(attention_weights_t)
                
                if sample:
                    inp = targets[t].squeeze()
                else:
                    inp = attentional_state_t.squeeze()
            attentional_states = torch.stack(attentional_states, dim=0)
            attention_weights = torch.cat(attention_weights, dim=0)
        logits = attentional_states
        return logits, state, attention_weights


class DecoderState(dict):

    def __init__(self, rnn_state, attentional_state=None,
                 input_feeding=False, decoder_layers=None):
        super().__init__()
        self['input_feeding'] = input_feeding
        self['rnn'] = rnn_state
        assert input_feeding or attentional_state is None
        self['attention'] = None
        if input_feeding:
            self['attention'] = attentional_state

    @staticmethod
    def apply_to_rnn_state(fn, rnn_state):
        if isinstance(rnn_state, tuple):  # LSTM
            return tuple(fn(s) for s in rnn_state)
        else:
            return fn(rnn_state)

    @property
    def input_feeding(self):
        return self['input_feeding']

    @property
    def rnn(self):
        return self['rnn']

    @property
    def attention(self):
        return self['attention']

    def prepForDecoder(self, num_layers):
        if isinstance(self["rnn"], tuple):
            # LSTM
            newState = []
            for i in range(2):
                numLayersTDirections, batch_size, hidden_dim = self["rnn"][i].size()
                separateDirections = self["rnn"][i].view(num_layers, -1, batch_size, hidden_dim)
                if separateDirections.size(1) == 2:
                    forwardHidden = separateDirections[:,0,:,:]
                    reverseHidden = separateDirections[:,1,:,:]
                    hidden = torch.cat([forwardHidden, reverseHidden], dim=2)
                else:
                    hidden = separateDirections[:,0,:,:]
                newState.append(hidden)
            self["rnn"] = (newState[0], newState[1])
        else:
            # GRU
            numLayersTDirections, batch_size, hidden_dim = self["rnn"].size()
            separateDirections = self["rnn"].view(num_layers, -1, batch_size, hidden_dim)
            if separateDirections.size(1) == 2:
                forwardHidden = separateDirections[:,0,:,:]
                reverseHidden = separateDirections[:,1,:,:]
                hidden = torch.cat([forwardHidden, reverseHidden], dim=2)
            else:
                hidden = separateDirections[:,0,:,:]
            self["rnn"] = hidden
        return separateDirections.size(1) == 2



    def update_state(self, rnn_state=None, attentional_state=None):
        self.update_rnn_state(rnn_state)
        self.update_attentional_state(attentional_state)

    def update_rnn_state(self, rnn_state):
        self['rnn'] = rnn_state

    def update_attentional_state(self, attentional_state):
        if self.input_feeding and attentional_state is not None:
            self['attention'] = attentional_state

    def repeat(self, beam_size):
        new_state = copy.copy(self)
        new_state['rnn'] = self.apply_to_rnn_state(
            fn=lambda s: s.repeat(1, beam_size, 1),
            rnn_state=new_state['rnn'])
        if self.input_feeding and new_state['attention'] is not None:
            new_state['attention'] = (
                new_state['attention'].repeat(beam_size, 1))
        return new_state

    def beam_update(self, batch_index, beam_indices, beam_size):
        def beam_update_fn(v):
            # The shape is (..., beam_size * batch_size, state_dim).
            orig_size = v.size()
            new_size = (
                orig_size[:-2]
                + (beam_size, orig_size[-2] // beam_size, orig_size[-1]))
            # beam_of_batch: (..., beam_size, state_dim)
            beam_of_batch = v.view(*new_size).select(-2, batch_index)
            beam_of_batch.data.copy_(
                beam_of_batch.data.index_select(-2, beam_indices))

        self.apply_to_rnn_state(fn=beam_update_fn, rnn_state=self['rnn'])
        if self.input_feeding and self['attention'] is not None:
            beam_update_fn(self['attention'])


In [ ]:
# %load ../batchedRNN/model/seq2seq.py
import torch
from torch import nn

# from . import encoders, decoders


class RecurrentSeq2Seq(nn.Module):

    def __init__(self, hidden_dim, dropout_prob,
                rnn_type, bidirectional, num_layers,
                attention_type, input_feeding,
                input_dim, output_dim, args):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout_prob = dropout_prob
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.attention_type = dropout_prob
        self.input_feeding = input_feeding
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.encoder = RecurrentEncoder(
            rnn_type=rnn_type, input_dim=input_dim,
            hidden_dim=hidden_dim, dropout_prob=dropout_prob,
            bidirectional=bidirectional, num_layers=num_layers)
        annotation_dim = hidden_dim
        if bidirectional:
            annotation_dim = hidden_dim * 2
        self.decoder = RecurrentDecoder(
            rnn_type=rnn_type, input_dim=output_dim, hidden_dim=hidden_dim,
            annotation_dim=annotation_dim, num_layers=num_layers,
            attention_type=attention_type, input_feeding=input_feeding,
            dropout_prob=dropout_prob, args=args)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.decoder.reset_parameters()

    def remove4D(self, encoder_inputs):
        inp0, inp1 = encoder_inputs[:,:,0,:], encoder_inputs[:,:,1,:]
        return torch.cat([inp0, inp1], dim=2)


    def forward(self, encoder_inputs, decoder_inputs, epoch=None):
        encoder_hidden_states, encoder_state = self.encoder(
            encoder_inputs=self.remove4D(encoder_inputs))
        decoder_state = DecoderState(
            rnn_state=encoder_state, input_feeding=self.input_feeding)
        logits, _, _ = self.decoder(
            annotations=encoder_hidden_states,
            targets=decoder_inputs, state=decoder_state, epoch=epoch)
        return logits


In [ ]:
import ast

class Bunch(object):
    def __init__(self, adict):
        self.__dict__.update(adict)
    def __str__(self):
        out = ""
        for k, v in self.__dict__.items():
            out += "{}: {}, ".format(k, v)
        return out
import json

In [ ]:
class PostProcess(object):
    def __init__(self, modelPath,args, chooseModel="rnn", dataDict=None):
        self.args = args
        self.modelPath = modelPath
        if dataDict:
            self.dataDict = dataDict
        else:
            self.dataDict = GetDataLoader("../data/traffic/trafficWithTime/")
        if chooseModel=="rnn":
            self.model = Seq2Seq(self.args)
        elif chooseModel=="sketch-rnn":
            self.model = SketchRNN(self.args)
        elif chooseModel == "Seq2SeqAttn":
            self.model = RecurrentSeq2Seq(
                self.args.h_dim,
                self.args.dropout_prob,
                self.args.rnn_type,
                self.args.bidirectionalEncoder,
                self.args.n_layers,
                self.args.attention_type,
                self.args.input_feeding,
                self.args.x_dim * args.channels,
                self.args.output_dim,
                self.args)
        self.model.eval()

    def getReconLoss(self, output, target):
        output = self.dataDict["scaler"].inverse_transform(output)
        target = self.dataDict["scaler"].inverse_transform(target)
        assert output.size() == target.size(), "output size: {}, target size: {}".format(output.size(), target.size())
        outputs = {}
        
        if args.criterion == "RMSE":
            mse = nn.MSELoss()
            loss = torch.sqrt(mse(output, target)).item()
        elif args.criterion == "L1Loss":
            l1loss = nn.L1Loss()
            loss = l1loss(output, target).item()
        return loss, output, target

    def sketchRNNKLD(self, latentMean, latentStd, trainingMode, epoch):
        LKL = -0.5*torch.sum(1+latentStd-latentMean**2-torch.exp(latentStd))\
                /float(args.z_dim*args.batch_size)
        if trainingMode:
            # update eta for LKL:
            eta_step = 1-(1-args.eta_min)*args.R**epoch
            if args.cuda:
                KL_min = torch.Tensor([args.KL_min]).cuda()
            else:
                KL_min = torch.Tensor([args.KL_min])
            return eta_step * torch.max(LKL, KL_min)
        else:
            return LKL

    def getLoss(self, output, target, epoch):
        if args.model == "rnn" or args.model == "Seq2SeqAttn":
            reconLoss, pred, target = self.getReconLoss(output, target)
            kldLoss = 0
            addtlDict = {}
        else:
            Pi, Mu, Sigma, latentMean, latentStd = output
            Pi = Pi.cpu().detach()
            Mu = Mu.cpu().detach()
            Sigma = Sigma.cpu().detach()
            latentMean = latentMean.cpu().detach()
            latentStd = latentStd.cpu().detach()
            reconLoss = self.sketchRNNReconLoss(target, Pi, Mu, Sigma)
            kldLoss = self.sketchRNNKLD(latentMean, latentStd, self.model.training, epoch)
            pred = self.dataDict["scaler"].inverse_transform(self.model.generatePred(Pi, Mu, Sigma))
            target = self.dataDict["scaler"].inverse_transform(target)
            addtlDict = {
                "Pi" : Pi,
                "Mu" : Mu,
                "Sigma": Sigma,
                "latentMean" : latentMean,
                "latentStd" : latentStd
            }
        outputDict = {
            "reconLoss" : reconLoss,
            "kldLoss" : kldLoss,
            "pred" : pred,
            "target" : target,
            "additional" : addtlDict
        }
        return outputDict

    def sketchRNNReconLoss(self, target, Pi, Mu, Sigma):
        stackedTarget = torch.stack([target] * Mu.size(3), dim=3)
        m = torch.distributions.Normal(loc=Mu, scale=Sigma)
        loss = torch.exp(m.log_prob(stackedTarget))
        loss = torch.sum(loss * Pi, dim=3)
        loss= -torch.log(loss)
        return loss.mean()

    def getEpochLoss(self, dataset, epoch):
        datas = []
        preds = []
        targets = []
        Pi = []
        Mu = []
        Sigma = []
        epoch_recon_val_loss = 0.0
        epoch_kld_val_loss = 0.0
        nValBatches = 0
        with torch.no_grad():
            for batchIDX, (inputData, target) in enumerate(map(self.dataDict["scaler"].transformBatchForEpoch, self.dataDict[dataset])):
                nValBatches += 1
                output = self.model(inputData, target, epoch)
                lossOutputs = self.getLoss(output, target, epoch)
                epoch_recon_val_loss += lossOutputs["reconLoss"]
                epoch_kld_val_loss += lossOutputs["kldLoss"]
                preds.append(lossOutputs["pred"].cpu().detach().numpy())
                targets.append(lossOutputs["target"].cpu().detach().numpy())
                datas.append(self.dataDict["scaler"].inverse_transform(inputData[:,:,0,:]).cpu().detach().numpy())
                if args.model == "sketch-rnn":
                    Pi.append(lossOutputs["additional"]["Pi"])
                    Mu.append(lossOutputs["additional"]["Mu"])
                    Sigma.append(lossOutputs["additional"]["Sigma"])
                
        datas = np.concatenate(datas, axis=0)
        preds = np.concatenate(preds, axis=0)
        targets = np.concatenate(targets, axis=0)
        if args.model == "sketch-rnn":
            Pi = np.concatenate(Pi, axis=1)
            Mu = np.concatenate(Mu, axis=1)
            Sigma = np.concatenate(Sigma, axis=1)
        retVals = {
            "reconLoss" : epoch_recon_val_loss / nValBatches,
            "kldLoss" : epoch_kld_val_loss / nValBatches,
            "preds" : preds,
            "targets" : targets,
            "datas" : datas,
            "Pi" : Pi,
            "Mu" : Mu,
            "Sigma" : Sigma
        }
        return retVals

    def prep(self, stateDictFile, dataset, epoch):
        desired_state_dict = torch.load(self.modelPath +stateDictFile, map_location=lambda storage, loc: storage)
        self.model.load_state_dict(desired_state_dict)
        self.model.eval()
        assert dataset in ["train", "val", "test"]
        

    def getLossAtEpoch(self, stateDictFile, dataset, epoch= -1):
        self.prep(stateDictFile, dataset, epoch)
        retVals = self.getEpochLoss(dataset, epoch)
        return retVals

In [ ]:
def plotTrainValCurve(trainLosses, valLosses, trainKLDLosses=None, valKLDLosses=None, labelTotal=False):
    plot_every = 1
    plt.rcParams.update({'font.size': 8})
    plt.figure()
    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Epoch")
    if args.model == "sketch-rnn":
        if labelTotal:
            ax1.set_ylabel("NLL + KLD Loss", color="r")
        else:
            ax1.set_ylabel("NLL", color="r")
    else:
        ax1.set_ylabel(args.criterion)
    ax1.tick_params('y')
    if labelTotal:
        labelT = "total train loss"
        labelV = "total validation loss"
    else:
        labelT = "train reconstruction loss"
        labelV = "validation reconstruction loss"
    ax1.plot(np.arange(1, len(trainLosses)+1)*plot_every, trainLosses, "r--", label=labelT)
    ax1.plot(np.arange(1, len(valLosses)+1)*plot_every, valLosses, color="red", label=labelV)
    ax1.legend(loc="upper left")
    ax1.grid()
    if trainKLDLosses:
        ax2 = ax1.twinx()
        ax2.set_ylabel("KLD Loss", color="b")
        ax2.tick_params('y', colors='b')
        ax2.plot(np.arange(1, len(trainKLDLosses)+1)*plot_every, trainKLDLosses, "b--", label="train KLD loss")
        ax2.plot(np.arange(1, len(valKLDLosses)+1)*plot_every, valKLDLosses, color="blue", label="val KLD loss")
        ax2.legend(loc="upper right")
        ax2.grid()
    plt.title("Losses for {}".format(args.model))

In [ ]:
def getDataForPreds(dataFile, batch_size, nHours=24):
    f = np.load(dataFile)
    startRow = np.random.choice(range(0, f["inputTimes"].shape[0] - 12*nHours))
    randomSensor = np.random.choice(range(0, f["inputs"].shape[2]))
    inputs = f["inputs"][startRow:startRow+12*nHours:12]
    targets = f["targets"][startRow:startRow+12*nHours:12]
    inputTimes = f["inputTimes"][startRow:startRow+12*nHours:12].reshape(-1,1)
    targetTimes = f["targetTimes"][startRow:startRow+12*nHours:12].reshape(-1,1)
    return inputs, targets, inputTimes, targetTimes, randomSensor

def makePreds(dataDict, model, batch_size, randomSensor, inputs, targets):
    nBatches = int(np.ceil(inputs.shape[0] / batch_size))
    datas, targetVals, preds = [],[],[]
    for batch in range(nBatches):
        if batch * (1 + batch_size) < inputs.shape[0]:
            inp = torch.FloatTensor(inputs[batch * batch_size : (1 + batch) * batch_size])
            target = torch.FloatTensor(targets[batch * batch_size : batch_size * (1 + batch)])
            stripLast = 0
        else:
            stripLast = 10 - inp_partial.shape[0]
            inp_partial = inputs[batch * batch_size:]
            target_partial = targets[batch * batch_size]
            other_input = np.stack([inp_partial[-1]] * stripLast, axis=0)
            other_target = np.stack([target_partial[-1]] * stripLast, axis=0)
            inp = torch.tensor(np.stack([inp_partial, other_input], axis=0))
            target = torch.tensor(np.stack([target_partial, other_target], axis=0))
        inp, target = dataDict["scaler"].transformBatchForEpoch((inp, target))
        model.eval()
        output = model(encoder_inputs = inp, decoder_inputs=target, epoch=0)
        output = dataDict["scaler"].inverse_transform(output).cpu().detach().numpy()
        inp = dataDict["scaler"].inverse_transform(inp[:,:,0,:]).cpu().detach().numpy()
        target = dataDict["scaler"].inverse_transform(target).cpu().detach().numpy()
        datas.append(inp[:inp.shape[0] - stripLast,:,randomSensor])
        targetVals.append(target[:target.shape[0] - stripLast,:,randomSensor])
        preds.append(output[:output.shape[0] - stripLast, :, randomSensor])
    return datas, targetVals, preds
    

In [ ]:
def plotHours(datas, targets, preds, inputTimes, targetTimes,dataset):
    f, ax = plt.subplots()
    f.set_figwidth(15)
    plt.plot(targetTimes, preds, label="pred")
    plt.plot(targetTimes, targets, label="target")
    plt.xticks(rotation=90)
    xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
    ax=plt.gca()
    ax.xaxis.set_major_formatter(xfmt)
    plt.legend()
    plt.ylabel("mile/h")
    N = preds.shape[0] / 12
    plt.title("{} Hour Sample Prediction {}".format(int(N), dataset))
    yMin = np.min((np.min(preds)-10, np.min(targets)-10, 10))
    yMax = np.max((np.max(preds)+10, np.max(targets)+10, 70))

In [ ]:
def plotNHours(preds, targets, datas, dataset, targetTimes, N=24):
    assert False, "Need to fix"
    instance = np.random.randint(targets.shape[1])
    sensor = np.random.randint(targets.shape[2])
    sequenceTrueMean = []
    sequenceTrueStd = []
    sequenceSampleMean = []
    sequenceSampleStd = []
    sequenceTarget = []
    sequenceTimes = []
    shouldMask = []
    maskindex = []
    lastTime = None
    for tStep in range(N):
        realIndex = instance + 12 * tStep
        if realIndex >= means.shape[1]:
            break
        if lastTime and inMinutes(targetTimes[realIndex, -1] - lastTime) > 5:
            shouldMask += [True]
        else:
            shouldMask += [False]
        lastTime = targetTimes[realIndex, -1]
        maskindex += [len(sequenceTrueMean)]
        m = means[:, realIndex, sensor]
        std = stds[:, realIndex, sensor]
        predSamples, sampleMean, sampleStd = getScaledSamples(m, std, dataMean, dataStd)
        sequenceTrueMean += list(m)
        sequenceTrueStd += list(std)
        sequenceSampleMean += list(sampleMean)
        sequenceSampleStd += list(sampleStd)
        sequenceTarget += list(targets[:, realIndex, sensor])
        sequenceTimes += list(targetTimes[realIndex])
        
    #f, ax = plt.subplots(2, sharex=True)
    #f.subplots_adjust(hspace=.5)
    """
    maskedSampleMean = ma.array(sequenceSampleMean)
    maskedTarget = ma.array(sequenceTarget)
    print(maskindex)
    print(shouldMask)
    print(maskedSampleMean.shape)
    for idx, should in zip(maskindex, shouldMask):
        if should:
            maskedSampleMean[idx] = ma.masked
            maskedTarget[idx] = ma.masked
    """
    #print(np.max(sequenceSampleStd), sequenceTimes[np.argmax(sequenceSampleStd)])
    #print(sequenceSampleMean)
    f, ax = plt.subplots()
    f.set_figwidth(15)
    plt.plot(sequenceTimes, sequenceSampleMean, label="pred")
    plt.plot(sequenceTimes, sequenceTarget, label="target")
    plt.fill_between(sequenceTimes,np.array(sequenceSampleMean)-1.96*np.array(sequenceSampleStd), np.array(sequenceSampleMean)+1.96*np.array(sequenceSampleStd), alpha=0.5)
    plt.xticks(rotation=90)
    xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
    ax=plt.gca()
    ax.xaxis.set_major_formatter(xfmt)
    plt.legend()
    plt.ylabel("mile/h")
    plt.title("{} Hour Sample Prediction {}".format(N, dataset))
    yMin = np.min((np.min(sequenceSampleMean)-10, np.min(sequenceTarget)-10, 10))
    yMax = np.max((np.max(sequenceSampleMean)+10, np.max(sequenceTarget)+10, 70))
    #plt.ylim((yMin,yMax))

In [ ]:
z = np.zeros((5,))
z[:z.shape[0] - 0]